# Example for the Memory Augmented Generator (MAG) model

## Imports + model initialization

In [ ]:
import pandas as pd
import tensorflow as tf

from loggers import set_level, add_handler
from models.nlu import infer_to_str
from models.qa import MAG
from utils import set_display_options
from datasets import get_dataset, prepare_dataset, test_dataset_time
from models import get_pretrained, get_models, get_model_history

from experiments_mag import config_from_name, training_config_from_name, testing_config_from_name, predict_config_from_name

set_display_options()

model_name = 'mag_nq_coqa_newsqa_split_off_wt_ib_2_2_mean'

print("Tensorflow version : {}".format(tf.__version__))

In [ ]:
config = config_from_name(model_name)
config.pop('class')

if 'pretrained_name' in config and not is_model_name(config['pretrained_name']):
    logging.warning('Pretrained model {} does not exist !'.format(config.pop('pretrained_name')))

tf.config.set_visible_devices([], 'GPU')
if 'pretrained_name' in config:
    model = MAG.from_pretrained(** config)
else:
    model = MAG(** config)

print(model)

In [ ]:
model.model.freeze(trainable = True)
model.encoder.subsampling_layer.trainable = False
model.summary()
model.model.encoder.summary()

## Training

In [ ]:
model = get_pretrained(model_name)

config = training_config_from_name(model_name, model.epochs > 0)

model.compile(** config.pop('compile_config', {}))

dataset = get_dataset(config['dataset'], ** config.pop('dataset_config', {}))

train, valid = dataset['train'], dataset['valid']

print("Training samples   : {} - {} batches".format(len(train), len(train) // config['batch_size']))
print("Validation samples : {} - {} batches".format(len(valid), len(valid) // (config['batch_size'] * 2)))

hist = model.train(
    train, validation_data = valid, valid_batch_size = 2., ** config
)

In [ ]:
model.plot_history()
print(model.history)

## Evaluate

In [ ]:
test_name = 'test'

model = get_pretrained(model_name)

config = testing_config_from_name(model_name, test_name, overwrite = False)

valid = get_dataset(config['dataset'], ** config.pop('dataset_config', {}), modes = 'valid')

hist = model.test(valid, ** config)

## Prediction

In [ ]:
pred_name = 'pred'

model = get_pretrained(model_name)

config = predict_config_from_name(model_name, pred_name, overwrite = False)

valid = get_dataset(config['dataset'], ** config.pop('dataset_config', {}), modes = 'valid')

hist = model.test(valid, ** config)

In [ ]:
question = [
    'How is the night vision of cat ?',
    'How is the night vision of cat ?',
    'What is the anoatomy of a cat ?',
    'How many paws does a cat have ?',
    'How many paws does a cat have ?',
    'How many paws does a cat have ?',
    'What is the origin of life ?'
]
context  = [
    'The cat is similar in anatomy to the other felid species: it has a strong flexible body, \
quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well \
developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-\
specific body language. A predator that is most active at dawn and dusk (crepuscular), the cat is a solitary hunter but a social species. \
It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals.[7] It secretes and \
perceives pheromones.',
    [p.strip() + '.' for p in 'The cat is similar in anatomy to the other felid species: it has a strong flexible body, \
quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well \
developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-\
specific body language. A predator that is most active at dawn and dusk (crepuscular), the cat is a solitary hunter but a social species. \
It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals.[7] It secretes and \
perceives pheromones.'.split('.') if len(p) > 0],
    ['The cat is similar in anatomy to the other felid species: it has a strong flexible body, \
quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well \
developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-\
specific body language. A predator that is most active at dawn and dusk (crepuscular), the cat is a solitary hunter but a social species. \
It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals.[7] It secretes and \
perceives pheromones.', 'The answer to everything is 42'],
    'A cat is an animal which has 4 paws and whiskers.',
    'A cat is an animal which has 4 paws and whiskers. However, everyone knows that the answer to everything is 42 !',
    ['A cat is an animal which has 4 paws and whiskers.', 'However, everyone knows that the answer to everything is 42 !'],
    'The answer to everything is 42.'
]

if not isinstance(question, list): question = [question]
if not isinstance(context, list): context = [context]

answers = model.predict(question, context, title = 'cat', method = 'beam')

for q, c, a in zip(question, context, answers):
    print("Question : {}\nContext : {}\nAnswer : {}\n".format(q, c, infer_to_str(a[0], a[1]))


## Tests

In [ ]:
#model.negative_mode = 'doc'
model = get_pretrained(model_name)

model.max_input_texts = 4
model.use_multi_input = False
model.max_input_length = 512
model.max_sentence_length = 128
model.merge_multi_inputs  = True

valid = get_dataset('nq', include_document = True, keep_mode = 'all', modes = 'valid')

config = model.get_dataset_config(batch_size = 16, is_validation = False, shuffle_size = 0)
ds = prepare_dataset(valid, ** config, is_rectangular = False)

test_dataset_time(ds, steps = 1000 if not model.use_multi_input else 10)

In [ ]:
from custom_train_objects.optimizers import WarmupScheduler, DivideByStep

lr = WarmupScheduler(maxval = 5e-5, minval = 5e-6, factor = 32, warmup_steps = 128)
lr = DivideByStep(0.5, maxval = 5e-5, minval = 5e-6)
lr.plot(50000)